### Use Neural Networks for test data and calculate accuracy and confidence

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: marcodia
"""
import numpy as np
import random
import xarray as xr
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
import tensorflow as tf

import import_ipynb
import sys
import os 

import network_arch as network
import metrics
import plot
import settings
import functions_misc as funcs 

import cftime
import matplotlib.pyplot as plt
import nc_time_axis

from cartopy import config
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from shapely.geometry.polygon import LinearRing

In [3]:
# <<< PARAMETERS >>>

num_experiments = 10
num_seeds = 10 
exp_num = 5

#confidence params
conf_level = 'most' #OR least
statement = 20      # or 80 
perc = 100-statement #percentile; this number can be stated as "the statement% conf_level confident predictions"; ex- the 20% most confident predictions  
subset = 'all'      # or correct, incorrect

running_window_yr = 10 #years of running mean
days_per_annualszn = 30+31+31+28 #number of days in months used; here Nov, Dec, Jan, Feb 
running_window = running_window_yr * days_per_annualszn

YEARS = '1850-1949'
STRT = pd.to_datetime('11-01-1850')
END   = pd.to_datetime('2-28-1949')  + dt.timedelta(days=1)

timeplot_full = np.arange(1850,1950)
array_size = len(timeplot_full) - running_window_yr
timeplot = timeplot_full[running_window_yr:(len(timeplot_full))]

NLABEL = 2

# >>>>>>>>>>>>>>>>>>>>>

In [4]:
# MAKE THE NN ARCHITECTURE
def make_model():
    # Define and train the model
    tf.keras.backend.clear_session()
    model = network.defineNN(HIDDENS,
                             input1_shape = X_train.shape[1],
                             output_shape=NLABEL,
                             ridge_penalty1=RIDGE1,
                             dropout=DROPOUT,
                             act_fun='relu',
                             network_seed=NETWORK_SEED)
    
    loss_function = tf.keras.losses.CategoricalCrossentropy()    
    model.compile(
                  optimizer = tf.keras.optimizers.Adam(learning_rate=LR_INIT),
                  loss = loss_function,
                  metrics = [
                      tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None),
                      metrics.PredictionAccuracy(NLABEL)
                      ]
                  )           
    return model, loss_function

In [37]:
for sub_exp in np.arange(exp_num*100,exp_num*100+10):
    EXPERIMENT = EXPERIMENT NAME #need to name from settings file

    Xdata_folder = X_DATA #data folder with x-data
    Ydata_folder = Y_DATA #data folder with y-data

    ddir_X = DIRECTORY_X
    ddir_Y = DIRECTORY_Y
    ddir_MODEL = DIRECTORY_MODEL #directory with trained neural network model
    ddir_out = DIRECTORY_OUT 

    params = settings.get_settings(EXPERIMENT)

    PREDICTOR_VAR  = params['PREDICTOR_VAR']           
    PREDICTAND_VAR = params['PREDICTAND_VAR']              
    REGION_TOR     = params['REGION_TOR']          
    REGION_TAND    = params['REGION_TAND']            
    training_ens   = params['training_ens']            
    validation_ens = params['validation_ens']           
    testing_ens    = params['testing_ens']           
    train_list     = params['train_list']           
    lead           = params['lead']            
    days_average   = params['days_average']            
    GLOBAL_SEED    = params['GLOBAL_SEED']            
    HIDDENS        = params['HIDDENS']          
    DROPOUT        = params['DROPOUT']            
    RIDGE1         = params['RIDGE1']                    
    LR_INIT        = params['LR_INIT']
    BATCH_SIZE     = params['BATCH_SIZE']           
    RANDOM_SEED    = params['RANDOM_SEED']            
    act_fun        = params['act_fun']            
    N_EPOCHS       = params['N_EPOCHS']           
    PATIENCE       = params['PATIENCE']   
    

#>>>>>SET UP <<<<<<<<<<<<<<<
    np.random.seed(GLOBAL_SEED)
    random.seed(GLOBAL_SEED)
    tf.compat.v1.random.set_random_seed(GLOBAL_SEED)
    
    #Re-process training, validation to use information from them; this code is the same as NeuralNetwork_train 
    
    time_range = xr.cftime_range(str(STRT)[:10], str(END)[:10],calendar = 'noleap') #[0:10] corresponds to full datestamp
    time_range_ndjf = time_range.where(funcs.is_ndjf(time_range.month)).dropna()
    TIME_X = xr.DataArray(time_range_ndjf + dt.timedelta(days=0), dims=['time'])     
    TIME_Y = xr.DataArray(time_range_ndjf + dt.timedelta(days=lead+days_average), dims=['time'])  
    
    # ----- X TRAINING ------
    X_finame = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+train_list+'_dailyanom_detrend.nc'
    X_all_full = xr.open_dataarray(ddir_X+X_finame)
    X_all = X_all_full.where(X_all_full.time == TIME_X, drop=True)


    Xtrain = X_all.stack(time_all=('ens','time')) # lat,lon,time*8 (8= number of training ens members) 
    Xtrain = Xtrain.transpose('time_all','lat','lon') # time*8,lat,lon

    Xtrain_std = np.std(Xtrain,axis=0)
    Xtrain_mean = np.mean(Xtrain,axis=0)
    Xtrain = (Xtrain-Xtrain_mean)/Xtrain_std
    X_train = Xtrain.stack(z=('lat','lon'))

    # ---------- X VALIDATION (and TESTING)----------
    X_finame  = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+str(validation_ens)+'_dailyanom_detrend.nc'
    Xval = xr.open_dataarray(ddir_X+X_finame)

    Xval= Xval.where(Xval.time == TIME_X, drop=True)
    Xval = (Xval - Xtrain_mean)/Xtrain_std
    print(Xval.time)
    
    X_TEST_finame  = PREDICTOR_VAR+'_'+REGION_TOR+'_'+YEARS+'_'+'ens'+str(testing_ens)+'_dailyanom_detrend.nc'
    Xtest = xr.open_dataarray(ddir_X+X_TEST_finame)

    Xtest = Xtest.where(Xtest.time == TIME_X, drop=True)
    Xtest = (Xtest - Xtrain_mean)/Xtrain_std

    #%% ----- Y TRAINING--------

    Ytrain_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_boxavg_'+YEARS+'_'+'ens'+train_list+'_dailyanom_detrend_14dayavg.nc'
    Y_all = xr.open_dataarray(ddir_Y+Ytrain_finame)
    Y_all= Y_all.where(Y_all.time == TIME_Y, drop=True)

    # ----- Standardize Y training -----
    Ytrain = Y_all[:,:]    #already box averaged so no lat or lon dimension 
    Ytrain = Ytrain.stack(time_all=('ens','time')) # time*8 (Ytrain time is actually 365*100-(13*8) because of 14-day average)
    Ytrain_med = np.median(Ytrain)
    Ytrain = Ytrain - Ytrain_med       #Subtracting the median forces the output above or below zero 

    # ----- Y VALIDATION --------
    Yval_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_boxavg_'+YEARS+'_'+'ens'+str(validation_ens)+'_dailyanom_detrend_14dayavg.nc'
    Yval_all  = xr.open_dataarray(ddir_Y+Yval_finame)
    Yval_all= Yval_all.where(Yval_all.time == TIME_Y, drop=True)

    # ----- Standardize Y validation -----
    Yval = Yval_all[:]
    Yval = Yval - Ytrain_med         
    print(Yval.time)
    # ----- Grab Y testing -----

    Ytest_finame = PREDICTAND_VAR+'_'+REGION_TAND+'_boxavg_'+YEARS+'_'+'ens'+str(testing_ens)+'_dailyanom_detrend_14dayavg.nc'
    Ytest_all  = xr.open_dataarray(ddir_Y+Ytest_finame)
    Ytest_all= Ytest_all.where(Ytest_all.time == TIME_Y, drop=True)

    Ytest = Ytest_all[:]
    Ytest = Ytest - Ytrain_med 

    # ----- Make binary -----
    # training
    Ytrain[np.where(Ytrain>=0)[0]] = 1
    Ytrain[np.where(Ytrain<0)[0]]  = 0
    # validation
    Yval[np.where(Yval>=0)[0]] = 1
    Yval[np.where(Yval<0)[0]]  = 0
    # testing
    Ytest[np.where(Ytest>=0)[0]] = 1
    Ytest[np.where(Ytest<0)[0]]  = 0

    # make Yval have equal 0s and 1s so that random chance is 50%
    n_valzero = np.shape(np.where(Yval==0)[0])[0]
    n_valone  = np.shape(np.where(Yval==1)[0])[0]
    i_valzero = np.where(Yval==0)[0]
    i_valone  = np.where(Yval==1)[0]

    if n_valone > n_valzero:
        isubset_valone = np.random.choice(i_valone,size=n_valzero,replace=False)
        i_valnew = np.sort(np.append(i_valzero,isubset_valone))
        Yval = Yval.isel(time=i_valnew,drop=True)
        X_val  = Xval[i_valnew].stack(z=('lat','lon'))
    elif n_valone < n_valzero:
        isubset_valzero = np.random.choice(i_valzero,size=n_valone,replace=False)
        i_valnew = np.sort(np.append(isubset_valzero,i_valone))
        Yval = Yval.isel(time=i_valnew,drop=True)
        X_val  = Xval[i_valnew].stack(z=('lat','lon'))
    else:
        X_val = Xval.stack(z=('lat','lon'))

    # ----- Make one hot vector -----
    enc = preprocessing.OneHotEncoder()
    onehotlabels      = enc.fit_transform(np.array(Ytrain).reshape(-1, 1)).toarray()
    onehotlabels_val  = enc.fit_transform(np.array(Yval).reshape(-1, 1)).toarray()
    onehotlabels_test  = enc.fit_transform(np.array(Ytest).reshape(-1, 1)).toarray()

    X_test_calcs = Xtest.stack(z=('lat','lon'))
    hotlabels_test_calcs = onehotlabels_test[:,:NLABEL]

    output_class_test = np.array(Ytest)
    output_test = (output_class_test.reshape(-1,1) == np.unique(output_class_test)).astype(int)
    calcpercent = lambda cat: str((np.sum(output_class_test == cat)/len(output_class_test)*100).astype(int))
    # Print out the sizes of each class
    print('Frequency for each Category')
    print('Below Zero: ' + calcpercent(0) + '%')
    print('Above Zero: ' + calcpercent(1) + '%')
     
    #>>>>>>>>Analyze test data on trained model<<<<<<<<
    NETWORK_SEED=0
    model, LOSS = make_model() 

    tf.get_logger().setLevel('ERROR')
    
    
    #Need to redefine empty matrices for each experiment
    #Code modified from Dr. Kirsten Mayer
    
    accuracy_test = np.zeros([array_size])
    stored_accuracy = np.zeros(len(RANDOM_SEED))
    ind = 0
    ind2 = 0
    acc_test = np.zeros(shape=(10,20)) + np.nan    #this is the shape of [# seeds, # of confidence intervals]
    acc_val = np.zeros(shape=(10,20)) + np.nan
    acc_moving = np.zeros(shape=(num_seeds,array_size)) + np.nan
    
    #<<<<< LOAD MODEL TO RUN CALCULATIONS ON PREDICTIONS >>>>>>>
    for SEED in RANDOM_SEED:
        if ind < num_experiments:
            model.load_weights(ddir_MODEL+'_operationalseed'+str(SEED)+'.h5')
            X1= X_test_calcs.copy(deep=True)
            
            #<<<Apply testing data to trained model>>>>>>>>>>>>> 
            Ptest = model.predict(X_test_calcs)    #predicted values on test data - softmax output of confidence 
            Cptest_pred = Ptest.argmax(axis=1)     #0,1 of predicted  
            Cttest_true = hotlabels_test_calcs.argmax(axis=1) #true values on validation 
                  
            conf_pred_val = model.predict(X_val)           # softmax output
            cat_pred_val  = np.argmax(conf_pred_val, axis = 1) # categorical output
            max_conf_val  = np.max(conf_pred_val, axis = 1)   # predicted category confidence
          
            results_test = model.evaluate(X_test_calcs,hotlabels_test_calcs,verbose = 2) #prints out model evaluation 
            stored_accuracy[ind] = results_test[2] #2 hardcoded to get the accuracy (categorical and prediction accuracy are the same here)

        # ----- EVALUATE MODEL -----  
            conf_pred_test = model.predict(X_test_calcs)           # softmax output
            cat_pred_test  = np.argmax(conf_pred_test, axis = 1) # categorical output
            max_conf_test  = np.max(conf_pred_test, axis = 1)   # predicted category confidence
            
            #<<<< Rank data into most confident predictions for testing and validation>>>>>>>
            
            for p,per in enumerate(np.arange(0,100,5)):
                if ind < num_seeds: 
                    i_cover_test = np.where(max_conf_test >= np.percentile(max_conf_test,per))[0]
                    icorr_test   = np.where(cat_pred_test[i_cover_test] == Ytest[i_cover_test])[0]
                    if len(i_cover_test) == 0:
                        acc_test[ind,p] = 0.
                    else:
                        acc_test[ind,p] = (len(icorr_test)/len(i_cover_test)) * 100   
        
        
            for p,per in enumerate(np.arange(0,100,5)):
                if ind < num_seeds: 
                    i_cover_val = np.where(max_conf_val >= np.percentile(max_conf_val,per))[0]
                    icorr_val   = np.where(cat_pred_val[i_cover_val] == Yval[i_cover_val])[0]
                    if len(i_cover_val) == 0:
                        acc_val[ind,p] = 0.
                    else:
                        acc_val[ind,p] = (len(icorr_val)/len(i_cover_val)) * 100
                
            #<<<<<<<<<<>>>>>>>>>>>>>
            
            #<<<<<<<<<<Take window of accuracy>>>>>>>>>>>
            for i in np.arange(0,len(Cptest_pred)-running_window_yr,(days_per_annualszn)):
                if (running_window+i) <= (len(Cptest_pred)):
                    modelcorr_test = Cptest_pred[i:running_window+i]==Cttest_true[i:running_window+i]
                    nmodelcorr_test = modelcorr_test[modelcorr_test].shape[0]
                    ntest_test = Cttest_true[i:running_window+i].shape[0]
                    index_test = int(i/days_per_annualszn)
                    if index_test<99:
                        accuracy_test[index_test] = 100*nmodelcorr_test/ntest_test
                        
            accuracy_test_xr = xr.DataArray(accuracy_test)
            filename = 'accuracy_testdata_'+str(running_window_yr)+'yr_runavg_exp'+EXPERIMENT[-3:]+'_seed'+str(SEED)+'.nc'
            #accuracy_test_xr.to_netcdf(ddir_out+filename, mode='w')
            
            #<<<<<<<<<<>>>>>>>>>>>>>
            
            #Calculate Accuracy for confident predictions 
            
            for i in np.arange(0,len(Cptest_pred)-running_window_yr,(days_per_annualszn)):
                 if (running_window+i) <= (len(Cptest_pred)):
            
                    max_conf  = np.max(Ptest[i:running_window+i], axis = 1)   # predicted category confidence
                    Cptest_pred_cut = Cptest_pred[i:running_window+i]
                    Ytest_cut = Ytest[i:running_window+i]
                    
                    if conf_level == 'most':
                        i_cover = np.where(max_conf >= np.percentile(max_conf,perc))[0]  #find X% most confident predictions 
                    if conf_level == 'least':
                        i_cover = np.where(max_conf <= np.percentile(max_conf,perc))[0]  #find X% least confident predictions 
                        
                    icorr   = np.where(Cptest_pred_cut[i_cover] == Ytest_cut[i_cover])[0]   #find where the confident predictions match the truth values (ie find correct); Ytest is the NOT onehotencoded Y-values 

                    index_test = int(i/days_per_annualszn)
                    
                    if index_test<99:
                        acc_moving[ind,index_test] = (len(icorr)/len(i_cover)) * 100                #calculate accuracy; ind is the network seed  
            
            
            acc_vs_conf_moving = xr.DataArray(acc_moving)
            filename_moving = str(statement)+'confidence_vs_accuracy_'+str(running_window_yr)+'yr_runavg_'+EXPERIMENT[-3:]+'_allseeds.nc'
            acc_vs_conf_moving.to_netcdf(ddir_out+filename_moving, mode='w',format='NETCDF4')
            
            #<<<<<<<<<<<Split most confident data into correct and incorrect>>>>>>>
            #softmax:
            
            max_logits = np.max(Ptest,axis=-1)
            if conf_level == 'most':
                i_cover = np.where(max_logits >= np.percentile(max_logits, perc))[0]
                
            if conf_level == 'least':
                i_cover = np.where(max_logits < np.percentile(max_logits, perc))[0]
                
                
            cat_true_conf = Cttest_true[i_cover]
            cat_pred_conf = Cptest_pred[i_cover]
            
            if subset == 'correct':
                #location of correct predictions
                icorr = np.where(Cptest_pred[i_cover] - Cttest_true[i_cover] == 0)[0]
                X1_subset = X1[i_cover][icorr] #index Xtest with indicies of CONFIDENT & CORRECT predictions; for output compositing, replace X1 with Y variable 
                X_subset_time = X1.time[i_cover][icorr] #index with indicies of CONFIDENT & CORRECT predictions
                Y_true_subset = cat_true_conf[icorr]
                Y_pred_subset = cat_pred_conf[icorr]
                time = X_test_calcs.time[icorr]
            
            if subset == 'incorrect': #confident but incorrect/wrong prediction
                #location of incorrect predictions
                i_incorr = np.where(Cptest_pred[i_cover] - Cttest_true[i_cover] != 0)[0]
                X1_subset = X1[i_cover][i_incorr] #index Xtest with indicies of INCORRECT predictions
                X_subset_time = X1.time[i_cover][i_incorr] #index with indicies of INCORRECT predictions
                Y_true_subset = cat_true_conf[i_incorr]
                Y_pred_subset = cat_pred_conf[i_incorr]
                time = X_test_calcs.time[i_incorr]
            
            if subset == 'all':    #all confident predictions (correct and incorrect) 
                i_incorr = np.where(Cptest_pred[i_cover] - Cttest_true[i_cover] <= 1)[0]
                X1_subset = X1[i_cover][i_incorr] #index Xtest with indicies of ALL predictions
                X_subset_time = X1.time[i_cover][i_incorr] #index with indicies of ALL predictions
                Y_true_subset = cat_true_conf[i_incorr]
                Y_pred_subset = cat_pred_conf[i_incorr]
                time = X_test_calcs.time[i_incorr]
            
            # >>>>> LOOP THROUGH EACH CLASS: Calc & DATA MAPS FOR THE CLASS >>>>>
            classes = np.arange(0,NLABEL)
            for c in classes:
                sample = X1_subset.copy()
                icat = np.where(Y_true_subset == c)[0]# will be the same category as predicted if subset = 'correct'
                time_class = time[icat]
                Data1 = X1_subset[icat]
                Data1map = Data1[:,:np.shape(X1_subset)[1]].unstack('z')

                # <<<<< Array prep for plotting and saving to netcdf <<<<<

                # >>>>> Convert from numpy to xarray to save >>>>>
                Data1map_xr = xr.DataArray(Data1map) #,           #this retains the time dimension; changes because # of samples changes 
  
                # <<<<< Convert from numpy to xarray to save <<<<<
                Data_mean = np.nanmean(Data1map_xr,axis=0)   #this is a map to composite; ie averaged over time 
                Data_mean_xr = xr.DataArray(Data_mean)
                Data1map_xr.to_netcdf(ddir_out+subset+'_'+str(statement)+'perc_'+conf_level+'confident_predictions_'+EXPERIMENT[-3:]+'_seed'+str(SEED)+'_class'+str(c)+'.nc')
                Data_mean_xr.to_netcdf(ddir_out+subset+'TIMEMEAN_'+str(statement)+'perc_'+conf_level+'confident_predictions_'+EXPERIMENT[-3:]+'_seed'+str(SEED)+'_class'+str(c)+'.nc')
        
        ind = ind+1 
        ind2 = ind2+1

        acc_vs_conf_test = xr.DataArray(acc_test)
        filename_test = 'confidence_vs_accuracy_TESTING_'+EXPERIMENT[-3:]+'_allseeds.nc'
        acc_vs_conf_test.to_netcdf(ddir_out+filename_test, mode='w')  
        
        acc_vs_conf_val = xr.DataArray(acc_val)
        filename_val = 'confidence_vs_accuracy_VALIDATION_'+EXPERIMENT[-3:]+'_allseeds.nc'
        acc_vs_conf_val.to_netcdf(ddir_out+filename_val, mode='w')
        
        stored_accuracy_xr = xr.DataArray(stored_accuracy)
        filename2 = 'overall_accuracy_testdata'+'_exp'+EXPERIMENT[-3:]+'_allseeds'+'.nc'
        stored_accuracy_xr.to_netcdf(ddir_out+filename2, mode='w')